<a href="https://colab.research.google.com/github/avantgarde-cy/Netflix_Analysis/blob/sh/SH_Genres_moviedata_0712.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 데이터 불러오기 

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
movie_data = pd.read_csv('MoviesOnStreamingPlatforms.csv', encoding = 'unicode_escape')
use_df = pd.read_csv('use_df_0711.csv', encoding = 'unicode_escape')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

## 1.데이터 확인

### 1.1 movie_data 확인

 (https://www.kaggle.com/nikhileshkos/recommended-ott-movies-shows-analysis) 

In [ ]:
movie_data.head(2)

#### **활용하지 않을 데이터 드랍 후 해당 데이터를 global_data로 정의**

In [ ]:
global_data = movie_data.drop(['Unnamed: 0','ID','Year','Age','RottenTomatoes','Type','Language','Directors','Runtime','Country'], axis = 1)
global_data.head(2)

#### **넷플릭스에 있는 영화만 추출 ( Netflix = 1 인 데이터 보여주기 )** - **총 3559개**

In [ ]:
netflix_data = global_data.drop(['Hulu','PrimeVideo','Disney'], axis = 1)

In [ ]:
netflix_data = netflix_data.loc[(netflix_data.Netflix >= 1)]
netflix_data.head(2)

#### **넷플릭스 데이터 안에서 MISSING DATA 확인**


In [ ]:
sns.heatmap(netflix_data.isnull())

In [ ]:
netflix_data.isnull().sum() 

In [ ]:
netflix_data = netflix_data.dropna(subset=['Genres'])

In [ ]:
sns.heatmap(netflix_data.isnull())

#### **Genres**

In [ ]:
netflix_data['Genres'].value_counts()

In [ ]:
def splitting(dataframe,col):
    result = dataframe[col].str.get_dummies(',')
    print('Done!')
    return result

In [ ]:
n_genres = splitting(netflix_data,'Genres')

In [ ]:
n_genres

In [ ]:
netflix_data['IMDb'].value_counts()

- 추가 데이터로 사용하려고 가져왔으나 사용하지 않아서 전처리까지만 진행함


### 1.2 use_df 확인 (Kaggle data + FLIXPATROL)


In [ ]:
use_df.info()

In [ ]:
use_df.head(2)

In [ ]:
use_df.dtypes

In [ ]:
use_df = use_df.rename(columns={"IMDb Score":'IMDb_Score', "IMDb Votes":"IMDb_Votes"})
use_df.head(2)

## 2.VALUE & IMDb Score 분석 

#### **해당 데이터에서 쓰고자 하는 컬럼 외에 다른 것들 드랍**

In [ ]:
IMDB_base = use_df.drop(['Unnamed: 0','COUNTRY','Tags','Country Availability','Languages','Hidden Gem Score', 'Runtime', 'Director', 'Writer', 'Actors','Metacritic Score','Awards Received','Awards Nominated For','Release Date','Production House','Runtime','Summary','View Rating','Rotten Tomatoes Score','Boxoffice','Netflix Release Date','Genre_all'], axis = 1)
IMDB_base

In [ ]:
sns.heatmap(IMDB_base.isnull())

In [ ]:
IMDB_base = IMDB_base.dropna(subset=['IMDb_Score'])
IMDB_base = IMDB_base.dropna(subset=['IMDb_Votes'])
IMDB_base

#### **형 변환**

In [ ]:
IMDB_base.dtypes

In [ ]:
IMDB_base['IMDb_Votes'] = IMDB_base['IMDb_Votes'].astype('int')

In [ ]:
IMDB_base['IMDb_Score'] = IMDB_base['IMDb_Score'].astype('int')

In [ ]:
IMDB_base.dtypes

#### **결측치 확인 및 제거**

In [ ]:
sns.heatmap(IMDB_base.isnull())

In [ ]:
# IMDB_base['VALUE'].unique()

In [ ]:
len(IMDB_base['VALUE'].unique())

In [ ]:
IMDB_base['VALUE'].value_counts()

#### **데이터 분포 확인**

In [ ]:
sns.boxplot(x=IMDB_base['IMDb_Score'])

평점은 5~7점대가 대부분

In [ ]:
sns.boxplot(x=IMDB_base['IMDb_Votes'])

In [ ]:
sns.boxplot(x=IMDB_base['VALUE'])

-> 탑 랭킹 콘텐츠를 제외하고 대부분의 뷰포인트는 낮은 편

In [ ]:
IMDB_base.plot.scatter(x="IMDb_Score",y="VALUE",s=20);

#### **Movie_base**

---- Type 중에서 Movie만 남기기**

In [ ]:
movie_base = IMDB_base.loc[(IMDB_base.Type == 'Movie')]
movie_base

In [ ]:
movie_base.isnull().sum() 

In [ ]:
movie_base.dtypes

#### **Movie 데이터에서 IMDb 와 VALUE(points)와 상관관계**

In [ ]:
corr = movie_base.corr()
corr

In [ ]:
sns.heatmap(corr)

In [ ]:
movie_base.plot.scatter(x="IMDb_Score",y="VALUE",s=20);

#### **tv_base**

---- Type 중에서 TV만 남기기** 

전처리 과정은 위와 동일

In [ ]:
tv_base = IMDB_base.loc[(IMDB_base.Type == 'TV')]
tv_base

In [ ]:
tv_base.dtypes

In [ ]:
tv_base = tv_base.dropna(subset=['IMDb_Votes'])
tv_base = tv_base.dropna(subset=['IMDb_Score'])
tv_base

In [ ]:
tv_base['IMDb_Score'] = tv_base['IMDb_Score'].astype('int')
tv_base

In [ ]:
tv_base['IMDb_Votes'] = tv_base['IMDb_Votes'].astype('int')
tv_base

In [ ]:
tv_base.dtypes

#### **TV shows 중 IMDb 와 VALUE(points)와 상관관계**

In [ ]:
corr = tv_base.corr()
corr

In [ ]:
sns.heatmap(corr)

In [ ]:
tv_base.plot.scatter(x="IMDb_Score",y="VALUE",s=20);

## 3.Genre 와 IMDb 살펴보기

#### Moive

In [ ]:
movie_genre = movie_base.groupby("GENRE")
movie_genre.first()

In [ ]:
movie_genre["GENRE"].unique()

In [ ]:
movie_genre.plot(kind='box', y="VALUE", figsize=(10,6),vert=False)

#### TV

In [ ]:
tv_genre = tv_base.groupby("GENRE")
tv_genre.first()

In [ ]:
tv_genre["GENRE"].unique()

In [ ]:
tv_genre.plot(kind='box', y="VALUE", figsize=(10,6), vert=False)